In [1]:
import GRBgenerator

Below is the fastcube class, simulations for burstcube that run really fast but still pretty accurate. What sorts of customizations should I include?

Sample, samples n, 

Keep that test feature: 
maybe have a bruteforce vs. quicker version, all about fullsky sims now though. 

Also need to remember how I treated past horizon spots, and what the goal should be for those. 

In [2]:
#The following cell contains the "FastCube" class. This is the simulation I hope to use to be able to run quicker simulations. 

import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s
import time as time

class FastCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = (float(input("Please enter the second tilt (deg) ")))
            self.tiltB = np.deg2rad(self.tiltB)
            self.tiltC = self.tiltA = np.deg2rad(dettilt)
            self.tiltD = self.tiltB
        
        self.zenith = [0 , 0]
        self.bg = background


        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, test=True):   #is this how I inherit? 
        start = time.time()
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
            An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 
        
        test : boolean 
            For sanity purposes, if the simulation seems to give unrealistic results, switching to test mode allows for much quicker sampling, allowing it easier to spot potential errors. 
        
        

        Returns
        ----------
        localizationerrors : array
            numpy array that contains the average localization uncertainty at each sky position. 
        
        Additionally, response2GRB will print the sky position it is currently sampling, along with the average offset of localizations at that spot. 
        
        """
        
        if test:
            sample = 1
            samples = 50  #times  per sky pos
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 0
            topA = 1000
            ntheta = 10   #over sky chi points
            nphi = 37
            nA = 100

        else:
            sample = len(GRB.sourceangs) 
            samples = 50 #times  per sky pos
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 400
            topA = 1000
            ntheta = 31   #over sky chi points
            nphi = 120
            nA = 12
        self.localizationerrors = []    
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:
                sepA=bf.angle(sourcexyz,self.normA)
                   # print("separation from A is " + str(np.rad2deg(sepA)))
                   #this check passes.  
               
                if sepA < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryA=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normA))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryA = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsA = dtheoryA + self.bg #another artifact, incl this background effect somewhere
                unccountsA = np.sqrt(countsA)
                detactualA = rand.gauss(countsA,unccountsA)  #there is a lot of noise, present, updating it now. 
                if detactualA-self.bg < 0:
                    detactualA = self.bg
                    
                detcountsA = detactualA
                
                sepB=bf.angle(sourcexyz,self.normB)
                   # print("separation from B is " + str(np.rad2deg(sepB)))
                   #this check passes.  
               
                if sepB < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryB=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normB))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryB = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsB = dtheoryB + self.bg #another artifact, incl this background effect somewhere
                unccountsB = np.sqrt(countsB)
                detactualB = rand.gauss(countsB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = self.bg
                    
                detcountsB = detactualB

                sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation from C is " + str(np.rad2deg(sepC)))
                   #this check passes.  
               
                if sepC < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryC=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normC))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryC = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsC = dtheoryC + self.bg #another artifact, incl this background effect somewhere
                unccountsC = np.sqrt(countsC)
                detactualC = rand.gauss(countsC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = self.bg
                    
                detcountsC = detactualC
                
                sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation from D is " + str(np.rad2deg(sepD)))
                   #this check passes.  
               
                if sepD < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryD=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normD))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryD = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsD = dtheoryD + self.bg #another artifact, incl this background effect somewhere
                unccountsD = np.sqrt(countsD)
                detactualD = rand.gauss(countsD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = self.bg
                    
                detcountsD = detactualD
                
                #coarse to fine optimization
                chiA = bf.quad_solver(detcountsA,self.normA,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiB = bf.quad_solver(detcountsB,self.normB,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiC = bf.quad_solver(detcountsC,self.normC,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiD = bf.quad_solver(detcountsD,self.normD,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                
                chisquared = np.add(np.add(chiA,chiB),np.add(chiC,chiD)) #adds it all up for total chi2
                
                #print("Chi squareds: " +str(chisquared))
                
                
                thetaloc, philoc, Aguess = bf.indexer(chisquared,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA)
                recvec = hp.ang2vec(np.deg2rad(thetaloc),np.deg2rad(philoc))
                locoffset = np.rad2deg(bf.angle(sourcexyz,recvec))
               # print("Loc offset = " + str(locoffset) + " deg")
                
                locunc.append(locoffset)
                loop +=1
            print("Avg loc offset = " + str(s.mean(locunc)) + " deg.")
            self.localizationerrors.append(s.mean(locunc))
        return self.localizationerrors

In [1]:
GRBtest = GRBgenerator.Sky(8,500)

NameError: name 'GRBgenerator' is not defined

In [4]:
fastguy = FastCube(1000,45,alternating=False)

In [5]:
fastguy.response2GRB(GRBtest,test=False)

Testing [ 5.85026706 45.        ]
Avg loc offset = 6.356055490588208 deg.
Testing [  5.85026706 135.        ]
Avg loc offset = 6.277396510104684 deg.
Testing [  5.85026706 225.        ]
Avg loc offset = 5.716097201497 deg.
Testing [  5.85026706 315.        ]
Avg loc offset = 6.340620345071198 deg.
Testing [11.71585239 22.5       ]
Avg loc offset = 6.3311305796010835 deg.
Testing [11.71585239 67.5       ]
Avg loc offset = 6.244078307093961 deg.
Testing [ 11.71585239 112.5       ]
Avg loc offset = 5.810519107215404 deg.
Testing [ 11.71585239 157.5       ]
Avg loc offset = 5.97259394615734 deg.
Testing [ 11.71585239 202.5       ]
Avg loc offset = 5.894613450279301 deg.
Testing [ 11.71585239 247.5       ]
Avg loc offset = 5.968725219161077 deg.
Testing [ 11.71585239 292.5       ]
Avg loc offset = 6.244995977619226 deg.
Testing [ 11.71585239 337.5       ]
Avg loc offset = 6.3591272892337 deg.
Testing [17.61243907 15.        ]
Avg loc offset = 6.507507157032204 deg.
Testing [17.61243907 45. 

Avg loc offset = 4.297664122551953 deg.
Testing [ 41.85879221 340.71428571]
Avg loc offset = 4.793387005752663 deg.
Testing [ 41.85879221 353.57142857]
Avg loc offset = 5.882954483010421 deg.
Testing [48.1896851  5.625    ]
Avg loc offset = 8.361505207680755 deg.
Testing [48.1896851 16.875    ]
Avg loc offset = 8.212736669466688 deg.
Testing [48.1896851 28.125    ]
Avg loc offset = 8.37484873949124 deg.
Testing [48.1896851 39.375    ]
Avg loc offset = 4.326346682550715 deg.
Testing [48.1896851 50.625    ]
Avg loc offset = 3.913744921351687 deg.
Testing [48.1896851 61.875    ]
Avg loc offset = 6.846311788782906 deg.
Testing [48.1896851 73.125    ]
Avg loc offset = 7.292926322172511 deg.
Testing [48.1896851 84.375    ]
Avg loc offset = 7.40291603796577 deg.
Testing [48.1896851 95.625    ]
Avg loc offset = 7.319539510733231 deg.
Testing [ 48.1896851 106.875    ]
Avg loc offset = 7.529534682137096 deg.
Testing [ 48.1896851 118.125    ]
Avg loc offset = 6.798458884296299 deg.
Testing [ 48.1

Avg loc offset = 6.448396637968572 deg.
Testing [ 65.37568165 180.        ]
Avg loc offset = 6.564150833391007 deg.
Testing [ 65.37568165 191.25      ]
Avg loc offset = 6.721822134305955 deg.
Testing [ 65.37568165 202.5       ]
Avg loc offset = 9.113465318388172 deg.
Testing [ 65.37568165 213.75      ]
Avg loc offset = 18.387635305498574 deg.
Testing [ 65.37568165 225.        ]
Avg loc offset = 22.263238916208262 deg.
Testing [ 65.37568165 236.25      ]
Avg loc offset = 26.532809709098053 deg.
Testing [ 65.37568165 247.5       ]
Avg loc offset = 8.59869116836065 deg.
Testing [ 65.37568165 258.75      ]
Avg loc offset = 7.453736859518971 deg.
Testing [ 65.37568165 270.        ]
Avg loc offset = 5.999220423872083 deg.
Testing [ 65.37568165 281.25      ]
Avg loc offset = 7.052972378569088 deg.
Testing [ 65.37568165 292.5       ]
Avg loc offset = 8.993195951965701 deg.
Testing [ 65.37568165 303.75      ]
Avg loc offset = 24.128888788398886 deg.
Testing [ 65.37568165 315.        ]
Avg loc o

Avg loc offset = 27.3104029595257 deg.
Testing [ 80.40593177 331.875     ]
Avg loc offset = 31.107432584441042 deg.
Testing [ 80.40593177 343.125     ]
Avg loc offset = 24.45606873918462 deg.
Testing [ 80.40593177 354.375     ]
Avg loc offset = 8.00707599107417 deg.
Testing [85.21980815  0.        ]
Avg loc offset = 11.42284142405111 deg.
Testing [85.21980815 11.25      ]
Avg loc offset = 17.80451152267969 deg.
Testing [85.21980815 22.5       ]
Avg loc offset = 37.59979784378473 deg.
Testing [85.21980815 33.75      ]
Avg loc offset = 29.463835674484702 deg.
Testing [85.21980815 45.        ]
Avg loc offset = 37.7683965176223 deg.
Testing [85.21980815 56.25      ]
Avg loc offset = 37.8824057660763 deg.
Testing [85.21980815 67.5       ]
Avg loc offset = 32.25529770626629 deg.
Testing [85.21980815 78.75      ]
Avg loc offset = 30.996326533127196 deg.
Testing [85.21980815 90.        ]
Avg loc offset = 5.604343652786963 deg.
Testing [ 85.21980815 101.25      ]
Avg loc offset = 27.76755089924

Avg loc offset = 47.15264629123235 deg.
Testing [ 99.59406823 163.125     ]
Avg loc offset = 41.23719102740098 deg.
Testing [ 99.59406823 174.375     ]
Avg loc offset = 58.42870655120233 deg.
Testing [ 99.59406823 185.625     ]
Avg loc offset = 56.82668928063771 deg.
Testing [ 99.59406823 196.875     ]
Avg loc offset = 31.55384471911351 deg.
Testing [ 99.59406823 208.125     ]
Avg loc offset = 38.04752731876413 deg.
Testing [ 99.59406823 219.375     ]
Avg loc offset = 33.68409025543442 deg.
Testing [ 99.59406823 230.625     ]
Avg loc offset = 51.64782441820588 deg.
Testing [ 99.59406823 241.875     ]
Avg loc offset = 46.221271068799 deg.
Testing [ 99.59406823 253.125     ]
Avg loc offset = 49.33317424269775 deg.
Testing [ 99.59406823 264.375     ]
Avg loc offset = 57.649358680916144 deg.
Testing [ 99.59406823 275.625     ]
Avg loc offset = 55.50450546452387 deg.
Testing [ 99.59406823 286.875     ]
Avg loc offset = 49.72243929911027 deg.
Testing [ 99.59406823 298.125     ]
Avg loc offse

Avg loc offset = 91.19621995626581 deg.
Testing [114.62431835 292.5       ]
Avg loc offset = 79.26821781129222 deg.
Testing [114.62431835 303.75      ]
Avg loc offset = 97.81878617031798 deg.
Testing [114.62431835 315.        ]
Avg loc offset = 104.59167928731989 deg.
Testing [114.62431835 326.25      ]
Avg loc offset = 82.48242109890307 deg.
Testing [114.62431835 337.5       ]
Avg loc offset = 86.08399385905118 deg.
Testing [114.62431835 348.75      ]
Avg loc offset = 91.25882218675342 deg.
Testing [120.      5.625]
Avg loc offset = 89.20663747661193 deg.
Testing [120.     16.875]
Avg loc offset = 96.10630102612772 deg.
Testing [120.     28.125]
Avg loc offset = 89.11378482379651 deg.
Testing [120.     39.375]
Avg loc offset = 103.34105605228139 deg.
Testing [120.     50.625]
Avg loc offset = 114.88909927565719 deg.
Testing [120.     61.875]
Avg loc offset = 98.68566481886072 deg.
Testing [120.     73.125]
Avg loc offset = 89.82787422644766 deg.
Testing [120.     84.375]
Avg loc offse

KeyboardInterrupt: 

In [ ]:
np.linspace(400,1000,12)

In [ ]:
np